<a href="https://colab.research.google.com/github/camlet0630/ESG_ranking_system/blob/main/embedding_v0_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [ ]:
import gc
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizerFast, AutoModel
device = torch.device('cuda', 0) if torch.cuda.is_available() else 'cpu'
print("＊Device:", device)
# BERT Tokenizer: 使用 'bert-base-chinese' 版本
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
# BERT Model: 使用 'bert-base-chinese' 版本
bert = AutoModel.from_pretrained('bert-base-chinese').to(device)

＊Device: cpu


# Raw Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
raw_data = pd.read_csv("/content/drive/MyDrive/news.csv")
raw_data.head(5)

In [ ]:
# 在 for loop 中根據條件刪除行, 刪除沒有 content 的 row
for index, row in raw_data.iterrows():
  if type(raw_data["Content"][index]) != str:
    raw_data = raw_data.drop(index)

# 將每個段落都加上標題
now = "now"
for index, row in raw_data.iterrows():
  if type(raw_data["Name"][index]) == str:
    now = raw_data["Name"][index]
  if type(raw_data["Name"][index]) != str:
    raw_data["Name"][index] = now

num_of_content = len(raw_data)
# 將 label data 轉為 "E", "S", "G", 創建欄位
lis = [0] * num_of_content
raw_data["E"] = lis
raw_data["S"] = lis
raw_data["G"] = lis

# 檢查 tag 中是否包含字串
import re
def check_substrings(main_string, substrings):
    for substring in substrings:
        pattern = re.compile(re.escape(substring))
        if pattern.search(main_string):
            return True
    return False
def is_E(main_string):
  substrings = ["空氣污染", "能源管理", "燃料管理", "產品包裝", "生物多樣性", "溫室氣體排放", "水及污染管理", "廢棄物處理"]
  #substrings = ["空氣污染"]
  return check_substrings(main_string, substrings)
def is_S(main_string):
  substrings = ["人權", "社區關係", "客戶福利", "勞工關係", "薪酬與福利", "多樣性與共融", "僱員健康安全"]
  return check_substrings(main_string, substrings)
def is_G(main_string):
  substrings = ["商業倫理", "物料採購", "競爭行為", "激勵措施", "供應鏈管理", "系統化管理", "意外及安全"]
  return check_substrings(main_string, substrings)

for i, row in raw_data.iterrows():
  if type(raw_data["Tag01"][i]) == str:
    if is_E(raw_data["Tag01"][i]):
      raw_data["E"][i] = 1
    if is_S(raw_data["Tag01"][i]):
      raw_data["S"][i] = 1
    if is_G(raw_data["Tag01"][i]):
      raw_data["G"][i] = 1

  if type(raw_data["Tag02"][i]) == str:
    if is_E(raw_data["Tag02"][i]):
      raw_data["E"][i] = 1
    if is_S(raw_data["Tag02"][i]):
      raw_data["S"][i] = 1
    if is_G(raw_data["Tag02"][i]):
      raw_data["G"][i] = 1

  if type(raw_data["Tag03"][i]) == str:
    if is_E(raw_data["Tag03"][i]):
      raw_data["E"][i] = 1
    if is_S(raw_data["Tag03"][i]):
      raw_data["S"][i] = 1
    if is_G(raw_data["Tag03"][i]):
      raw_data["G"][i] = 1

  if type(raw_data["Tag04"][i]) == str:
    if is_E(raw_data["Tag04"][i]):
      raw_data["E"][i] = 1
    if is_S(raw_data["Tag04"][i]):
      raw_data["S"][i] = 1
    if is_G(raw_data["Tag04"][i]):
      raw_data["G"][i] = 1

news_articles = raw_data["Name"]+raw_data["Content"]
label_data = raw_data.iloc[:,-3:]

context_array = news_articles.values

In [ ]:
print(label_data)
label_data.to_csv('label_data.csv', index=False)

      E  S  G
0     0  0  0
1     0  0  0
2     0  0  0
3     0  0  0
4     0  0  0
...  .. .. ..
2080  0  0  0
2081  0  0  0
2082  1  0  0
2083  1  0  0
2084  0  0  0

[1805 rows x 3 columns]


In [ ]:
context_array
count = 0
for i in range(context_array.shape[0]):
    context = context_array[i]
    if (type(context) != str):
        print(type(context), ", i =", i)
        count += 1

print(count) # 280

0


# Bert Embedding


In [ ]:
def get_embedding_tensor(context_array):
    print("＊context_array Shape:", context_array.shape)
    embedding_tensor = torch.ones((context_array.shape[0], 768)) # 資料數 x 768
    for i in range(context_array.shape[0]):
        context = context_array[i]
        # 填充到 512, 截斷從 512, 返回 tensors
        bert_input = tokenizer(context, max_length=512, padding='max_length', truncation=True, return_tensors="pt")
        input_ids = bert_input['input_ids'].squeeze(1).to(device)
        attention_mask = bert_input['attention_mask'].to(device)
        final_inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}
        outputs = bert(**final_inputs)

        pooler_output = outputs.last_hidden_state[0][0].reshape(768)
        embedding_tensor[i] = pooler_output.detach().cpu()
        gc.collect()
        torch.cuda.empty_cache()

    return embedding_tensor

# 計算每個樣本各自的 BERT Embedding
embedding_tensor = get_embedding_tensor(context_array)

＊context_array Shape: (1805,)


In [ ]:
# 儲存 BERT Embedding Tensor
torch.save(embedding_tensor, "embedding_tensor.pt")


FileNotFoundError: ignored

In [ ]:
# 載入 BERT Embedding Tensor
embedding_tensor = torch.load("/content/embedding_tensor.pt")
print(embedding_tensor.shape)

torch.Size([1805, 768])
